# Laboratorio 4: Detección de Movimiento y Seguimiento de Objetos

**Asignatura:** Visión por Ordenador I  
**Grado:** Ingeniería Matemática e Inteligencia Artificial  
**Curso:** 2024/2025  

### Objetivo
En esta práctica, aprenderás a implementar algoritmos de detección de movimiento mediante sustracción de fondo y flujo óptico, y explorarás el filtro de Kalman para el seguimiento de objetos.

## Materiales
- **Python 3.8+**
- **OpenCV**: Puedes instalarlo con `pip install opencv-python`
- **Dataset de video**: Se usará un archivo de video o la cámara en tiempo real para probar los métodos de detección de movimiento.

In [1]:
import cv2
import os
import numpy as np

## Apartado A: Sustracción de Fondo

### Tarea A.1: Carga de Video
Carga un video en el cual se detectarán objetos en movimiento. Puedes utilizar
un video local o la cámara en tiempo real.

In [2]:
#TODO: Create a method that reads a video file (using VideoCapture from OpenCV) and returns its frames along with video properties
def read_video(videopath):
    """
    Reads a video file and returns its frames along with video properties.

    Args:
        videopath (str): The path to the video file.

    Returns:
        tuple: A tuple containing:
            - frames (list): A list of frames read from the video.
            - frame_width (int): The width of the video frames.
            - frame_height (int): The height of the video frames.
            - frame_rate (float): The frame rate of the video.
    """

    #TODO: Complete this line to read the video file
    cap = cv2.VideoCapture(videopath) 
    #TODO: Check if the video was successfully opened
    if not cap.isOpened():
        print('Error: Could not open the video file')

    #TODO: Get the szie of frames and the frame rate of the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Get the width of the video frames
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Get the height of the video frames
    frame_rate = cap.get(cv2.CAP_PROP_FPS) # Get the frame rate of the video
    
    #TODO: Use a loop to read the frames of the video and store them in a list
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames, frame_width, frame_height, frame_rate

#TODO: Path to the video file (visiontraffic.avi)
videopath = os.path.join('../data',  'visiontraffic.avi')


frames, frame_width, frame_height, frame_rate = read_video(videopath)

### Tarea A.2: Sustración de Fondo mediante diferencia de frames
Realiza una sustracción de fondo mediante diferencia de frames, para ello guarda
un frame con el fondo estático y úsalo como frame de referencia de fondo.

In [3]:
#TODO:  Show the frames to select the reference frame, press 'n' to move to the next frame and 's' to select the frame
for i, frame in enumerate(frames):
    #TODO: Show the frame
    cv2.imshow('Video', frame)
    # Wait for the key
    key = cv2.waitKey(0)
    # If the key is 'n' continue to the next frame
    if key == ord('n'):
        continue
    # If the key is 's' select the frame as the reference frame
    elif key == ord('s'):
        #TODO: Copy the frame to use it as a reference
        reference_frame = frame.copy()
        #TODO: Convert the reference frame to grayscale
        reference_frame = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)
        print('Frame {} selected as reference frame'.format(i))
        break

cv2.destroyAllWindows()

#TODO: Compute the difference between the reference frame and the rest of the frames and show the difference
for frame in frames:
    # Convert the frame to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #TODO: Compute the difference between the reference frame and the current frame
    diff = cv2.absdiff(reference_frame, frame)
    cv2.imshow('Diferencia', diff)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

Frame 0 selected as reference frame


### Tarea A.3: Configuración de la sustración de fondo con GMM
Configura el sustractor de fondo usando el modelo de mezcla de gaussianas
adaptativas (MOG2).

In [4]:
#TODO: Use MOG2 to detect the moving objects in the video

history = 500 # Number of frames to use to build the background model
varThreshold = 16  # Threshold to detect the background
detectShadows = True  # If True the algorithm detects the shadows

#TODO: Create the MOG2 object
mog2 = cv2.createBackgroundSubtractorMOG2(history, varThreshold, detectShadows)

### Tarea A.3: Aplicación de la Sustracción de Fondo

Aplica la sustracción de fondo en cada frame para extraer los objetos en movimiento.

In [5]:
#TODO: Use a loop to detect the moving objects in the video using the MOG2 algorithm and 
# save a video storing the parameters at the name of the file

#TODO: Create a folder to store the videos
output_folder = 'output_videos'
folder_path = os.path.join('../data', output_folder)
if not os.path.exists(folder_path):
    os.makedirs(folder_path, exist_ok=True)

#TODO: Name of the output video file with the parameters (history, varThreshold, detectShadows)
videoname = f'../data/output_videos/output_{history}_{varThreshold}_{detectShadows}.avi' # Name of the output video file with the parameters


#TODO: Create a VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'MJPG') # Codec to use
frame_size = (frame_width, frame_height) # Size of the frame
fps = frame_rate # Frame rate of the video
out = cv2.VideoWriter(videoname, fourcc, frame_rate, frame_size)

# masks = []
for frame in frames:
    #TODO: Apply the MOG2 algorithm to detect the moving objects
    mask = mog2.apply(frame)
    #TODO: Convert to BGR the mask to store it in the video
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    #TODO: Save the mask in a video
    # masks.append(mask)
    out.write(mask)

out.release()

**Preguntas del Apartado A**
1. ¿Cómo afecta la variable `varThreshold` a la precisión de la detección?
2. ¿Qué ventajas presenta `createBackgroundSubtractorMOG2` frente a métodos simples de diferencia de imágenes?

A.1
varThreshold controla la sensibilidad para detectar movimiento; valores bajos aumentan detección pero generan ruido, valores altos reducen sensibilidad.

A.2
createBackgroundSubtractorMOG2 se adapta a cambios de iluminación y fondos complejos, a diferencia de la simple diferencia de imágenes.


## Apartado B: Flujo Óptico

### Tarea B.1: Configuración del Flujo Óptico

Consulta la documentación de cv2.calcOpticalFlowPyrLK para ver que parametros se deben definir para realizar el calculo del flujo óptico

In [6]:
#TODO: Use the method to read the video file (slow_traffic_small.avi)
videopath = os.path.join('../data',  'slow_traffic_small.mp4')


videopath = videopath  # Path to the video file
frames, frame_width, frame_height, frame_rate = read_video(videopath)

#TODO: Define the parameters for Lucas-Kanade optical flow
winSize=(15, 15)
maxLevel= 2
criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)



### Tarea B.2: Detección de Puntos de Interés

Detecta los puntos de interés iniciales usando el algoritmo de Shi-Tomasi (cv2.goodFeaturesToTrack) en el primer frame.

In [7]:
#TODO: Detect the initial points of interest in the first frame

#TODO: Convert the first frame to grayscale
prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
# prev_gray = prev_gray.astype(np.float32)


#TODO: Define the parameters of the Shi-Tomasi algorithm
mask = None
maxCorners = 100
qualityLevel = 0.3
minDistance = 7
blockSize = 7

# Use the function goodFeaturesToTrack to detect the points of interest
p0 = cv2.goodFeaturesToTrack(prev_gray, mask=mask, maxCorners=maxCorners, qualityLevel=qualityLevel, minDistance=minDistance, blockSize=blockSize)

### Tarea B.3: Cálculo y Visualización del Flujo Óptico

In [8]:
#TODO: Use a loop to track the points of interest in the rest of the frames

# Create a mask image for drawing purposes
mask = np.zeros_like(frame)

for i, frame in enumerate(frames[1:]):
    #TODO: Copy the frame
    input_frame = frame.copy()
    # Convert the frame to grayscale
    frame_gray = cv2.cvtColor(input_frame, cv2.COLOR_BGR2GRAY)
    #TODO: Calculate the optical flow using the Lucas-Kanade algorithm

    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None, winSize=winSize, maxLevel=maxLevel, criteria=criteria)

    # Select the points that were successfully tracked
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        input_frame = cv2.circle(input_frame, (a, b), 5, (0, 0, 255), -1)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)

    #TODO: Update the inputs for the next iteration
    prev_gray = frame_gray # Copy the current frame to the previous frame
    p0 = p1 # Update the points to track
    # Show the frame with the tracks
    cv2.imshow('Frame', cv2.add(input_frame, mask))
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

**Preguntas del Apartado B**
1. ¿Qué efecto tiene el parámetro `winSize` en la precisión del flujo óptico?
2. ¿Cómo influye el parámetro `qualityLevel` en la función `cv2.goodFeaturesToTrack` al detectar puntos de interés?

B.1
winSize ajusta la precisión del flujo óptico: tamaños grandes mejoran precisión, pero aumentan el costo computacional.

B.2
qualityLevel define el mínimo de calidad para puntos de interés; valores altos mejoran precisión pero reducen cantidad de puntos.

## Apartado C: Filtro de Kalman para Seguimiento de Objetos

### Tarea C.1: Configuración del Filtro de Kalman

Inicializa el filtro de Kalman (cv2.KalmanFilter) con una matriz de medición y transición adecuada para un seguimiento en dos dimensiones.

In [9]:
#TODO: Create a method that reads a video file (using VideoCapture from OpenCV) and returns its frames along with video properties
def read_video(videopath):
    """
    Reads a video file and returns its frames along with video properties.

    Args:
        videopath (str): The path to the video file.

    Returns:
        tuple: A tuple containing:
            - frames (list): A list of frames read from the video.
            - frame_width (int): The width of the video frames.
            - frame_height (int): The height of the video frames.
            - frame_rate (float): The frame rate of the video.
    """

    #TODO: Complete this line to read the video file
    cap = cv2.VideoCapture(videopath) 
    #TODO: Check if the video was successfully opened
    if not cap.isOpened():
        print('Error: Could not open the video file')

    #TODO: Get the szie of frames and the frame rate of the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Get the width of the video frames
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Get the height of the video frames
    frame_rate = cap.get(cv2.CAP_PROP_FPS) # Get the frame rate of the video
    
    #TODO: Use a loop to read the frames of the video and store them in a list
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames, frame_width, frame_height, frame_rate


In [ ]:
#TODO: Use the method to read the video file (slow_traffic_small.avi)
videopath = os.path.join('../data',  'slow_traffic_small.mp4')
frames, frame_width, frame_height, frame_rate = read_video(videopath)

# TODO: Create the Kalman filter object
kf = cv2.KalmanFilter(4, 2)  # 4 estados (x, y, vx, vy) y 2 mediciones (x, y)

# TODO: Initialize the state of the Kalman filter
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)  # Matriz de medición 2x4
kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)  # Matriz de transición 4x4
kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2  # Covarianza de ruido de proceso 4x4

measurement = np.array((2, 1), np.float32)
prediction = np.zeros((2, 1), np.float32)

# TODO: Show the frames to select the initial position of the object
for i, frame in enumerate(frames):
    # Show the frame
    cv2.imshow('Frame', frame)
    # Wait for the key
    key = cv2.waitKey(0)
    # If the key is 'n' continue to the next frame
    if key == ord('n'):
        continue
    # If the key is 's' select the position of the object
    elif key == ord('s'):
        # Select the position of the object
        x, y, w, h = cv2.selectROI('Frame', frame, False)
        track_window = (x, y, w, h)
        
        # TODO: Compute the center of the object
        cx = x + w // 2
        cy = y + h // 2
        
        # TODO: Initialize the state of the Kalman filter
        kf.statePost = np.array([[cx], [cy], [0], [0]], np.float32)

        # Initialize the covariance matrix
        kf.errorCovPost = np.eye(4, dtype=np.float32)
        
        # Predict the position of the object
        prediction = kf.predict()
        
        # TODO: Update the measurement and correct the Kalman filter
        measurement = np.array([[cx], [cy]], np.float32)
        kf.correct(measurement)

        # TODO: Crop the object
        crop = frame[y:y+h, x:x+w].copy()
        
        # TODO: Convert the cropped object to HSV
        hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        
        # TODO: Compute the histogram of the cropped object (Reminder: Use only the Hue channel (0-180))
        crop_hist = cv2.calcHist([hsv_crop], [0], mask=None, histSize=[180], ranges=[0, 180])
        cv2.normalize(crop_hist, crop_hist, 0, 255, cv2.NORM_MINMAX)
        
        print(f'Initial position selected: {x}, {y}')
        break

cv2.destroyAllWindows()

Initial position selected: 330, 211


### Tarea C.2: Predicción y Corrección del Estado

Realiza la predicción del estado y corrige la posición estimada en cada iteración.

In [11]:
#TODO: Use the Kalman filter to predict the position of the points of interest

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 1)

for frame in frames[i:]:
    #TODO: Copy the frame 
    input_frame = frame.copy()
    #TODO: Convert the frame to HSV
    img_hsv = cv2.cvtColor(input_frame, cv2.COLOR_BGR2HSV)
    
    # Compute the back projection of the histogram
    img_bproject = cv2.calcBackProject([img_hsv], [0], crop_hist, [0, 180], 1)
    
    # Apply the mean shift algorithm to the back projection
    ret, track_window = cv2.meanShift(img_bproject, track_window, term_crit)
    x_,y_,w_,h_ = track_window
    #TODO: Compute the center of the object
    c_x = x_ + w_ // 2
    c_y = y_ + h_ // 2
    
    # Predict the position of the object
    prediction = kf.predict()

    #TODO: Update the measurement and correct the Kalman filter
    measurement = np.array([[c_x], [c_y]], np.float32)
    kf.correct(measurement)

    
    # Draw the predicted position
    cv2.circle(input_frame, (int(prediction[0][0]), int(prediction[1][0])), 5, (0, 0, 255), -1)
    cv2.circle(input_frame, (int(c_x), int(c_y)), 5, (0, 255, 0), -1)
    cv2.rectangle(input_frame, (x_, y_), (x_ + w_, y_ + h_), (255, 0, 0), 2)
    
    # Show the frame with the predicted position
    cv2.imshow('Frame', input_frame)
    key = cv2.waitKey(0)
    if key == ord('q'):
        break

cv2.destroyAllWindows()


**Preguntas del Apartado C**
1. ¿Cómo afecta el valor de `transitionMatrix` a la predicción en el filtro de Kalman?
2. ¿Cuál es la diferencia entre `measurementMatrix` y `transitionMatrix` en el contexto del seguimiento de objetos?

C.1
transitionMatrix afecta la predicción de movimiento del objeto, influyendo en la precisión del seguimiento en Kalman.
Con una transitionMatrix de 0.5 la predicción de movimiento resulta muy equivocada. Si ponemos un valor de 0.8, la predicción sigue un movimiento correcto y se asemeja a la real pero con cierto desfase inicial.

C.2
measurementMatrix ajusta el estado con mediciones, mientras transitionMatrix predice el movimiento del objeto.

## Ejercicio Adicional: Exploración del Modelo de Mezcla de Gaussianas (GMM)

**Objetivo**: Investiga cómo funciona el modelo de mezcla de gaussianas (GMM) para mejorar la detección en condiciones de iluminación cambiantes.

1. Implementación del GMM: Utiliza `cv2.createBackgroundSubtractorMOG()` y ajusta el parámetro `history` para observar cómo cambia la detección en función de la duración de la memoria del fondo.
2. Comparación con `MOG2`: Observa las diferencias en la sensibilidad a las sombras y los cambios de iluminación. Prueba con videos que incluyan cambios graduales de iluminación y objetos que se detienen temporalmente.

In [21]:
videopath = os.path.join('../data',  'slow_traffic_small.mp4')
cap = cv2.VideoCapture(videopath)

history = 10
history2 = 300

background_subtractor = cv2.bgsegm.createBackgroundSubtractorMOG(history=history, nmixtures=5, backgroundRatio=0.7, noiseSigma=0)
background_subtractor2 = cv2.bgsegm.createBackgroundSubtractorMOG(history=history2, nmixtures=5, backgroundRatio=0.7, noiseSigma=0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Aplicar el sustractor de fondo al fotograma
    fg_mask = background_subtractor.apply(frame)
    fg_mask2 = background_subtractor2.apply(frame)

    # Mostrar el resultado
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)
    cv2.imshow('Foreground Mask 2', fg_mask2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
videopath = os.path.join('../data',  'slow_traffic_small.mp4')
cap = cv2.VideoCapture(videopath)

history = 200

mog = cv2.bgsegm.createBackgroundSubtractorMOG(history=history)
mog2 = cv2.createBackgroundSubtractorMOG2(history=history, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    fg_mask_mog = mog.apply(frame)
    fg_mask_mog2 = mog2.apply(frame)

    # Mostrar los resultados
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask MOG', fg_mask_mog)
    cv2.imshow('Foreground Mask MOG2', fg_mask_mog2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


**Preguntas del Ejercicio Adicional**
1. ¿Qué ventajas observas en `createBackgroundSubtractorMOG` en comparación con `createBackgroundSubtractorMOG2`?
2. ¿Cómo afecta el parámetro `history` al rendimiento de detección en escenas con objetos que aparecen y desaparecen?

In [24]:
videopath = os.path.join('../data/test_videos',  'vtest.avi')
cap = cv2.VideoCapture(videopath)

history = 200

mog = cv2.bgsegm.createBackgroundSubtractorMOG(history=history)
mog2 = cv2.createBackgroundSubtractorMOG2(history=history, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    fg_mask_mog = mog.apply(frame)
    fg_mask_mog2 = mog2.apply(frame)

    # Mostrar los resultados
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask MOG', fg_mask_mog)
    cv2.imshow('Foreground Mask MOG2', fg_mask_mog2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


D.1
createBackgroundSubtractorMOG es más sencillo y consume menos recursos, pero MOG2 maneja sombras y cambios de iluminación con mayor precisión.
EL BackgroundSubtractorMOG2 hace un mucho mejor trabajo detectando las sombras y las diferencias de iluminación del vídeo. Es más preciso y detecta como frente los elementos de manera más concisa. También hay que tener en cuenta que ante los cambios de iluminación puede provocar un 'fogonazo' de píxeles que asume como frente cuando no lo son. Al paso del tiempo, acaba absorbiéndolas y añadiéndoselas al fondo aunque de manera lenta. El  BackgroundSubtractorMOG no tiene esta clase de problemas tal y como se puede observar en el video de comparación que se ejecuta en la celda anterior.  

D.2
El parámetro history afecta la duración de la memoria del fondo; valores altos mejoran detección de objetos estables, pero ralentizan la respuesta a cambios rápidos.
Se ha implementado un apartado dónde se generan 2 background substractors. Uno con 10 y otro con 300 de history. Aquél con 10 de histórico se actualiza mucho mas rápido que aquél de 300 y acaba absorbiendo cosas como si fuesen fondo (pintándolas en negro) mucho antes que aquél con un histórico mayor.